In [1]:
import os
import math
import numpy as np
from keras.models import Sequential  # NN Activation
from keras.layers import Conv2D  # Convolution Operation
from keras.layers import MaxPooling2D # Pooling
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense # Fully Connected Networks

Using TensorFlow backend.
c:\users\bigshot\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\bigshot\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\bigshot\miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\bigshot\miniconda3\lib\site-packages\tensorflow\python\framewo

In [11]:
import pandas as pd

In [ ]:
from argparse import ArgumentParser

In [3]:
import re
text = '1 +++$+++ are wtf ... awww thanks !'

In [10]:
text.split(maxsplit=2)

['1', '+++$+++', 'are wtf ... awww thanks !']

In [23]:
# Pre-processing: reading and parsing data
file_training_label = 'data/training_label.txt'
file_training_unlabel = 'data/training_nolabel.txt'

In [35]:
with open(file_training_label, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    sentiment = []
    text = []
    for line in lines:
        parsed = line.split(maxsplit=2)
        sentiment.append(int(parsed[0]))
        text.append(parsed[2].rstrip('\n'))
        
    data_training_label = pd.DataFrame({'sentiment': sentiment, 'text': text})

with open(file_training_unlabel, 'r', encoding='utf-8') as f:
    lines = f.readlines()    
    lines = [line.rstrip('\n') for line in lines]
    
    data_training_unlabel = pd.DataFrame({'text': lines})

In [36]:
# Improvement ideas
data_training_unlabel

,text
0,mkhang mlbo . dami niang followers ee . di q r...
1,don ' t you hate it when you hang on to a seem...
2,ok so never went to the movies because friend ...
3,can ' t wait to see diversity ' s performance !
4,i love britney spears haha joey this is what u...
5,wish i could call in but i can ' t do blogtalk...
6,1 more day !
7,nursing celeste with a tummy ache .
8,hates being this burnt !! ouch
9,just couldn ' t sleep last night . working 7a ...


In [ ]:
# Word embedding method 1: BOW(bag of words)
# Use Gensim module

In [ ]:
# Word embedding method 2: RNN

In [ ]:
# Semi supervised learning

In [ ]:
# Plot learning curve

In [ ]:
# Saving model

In [ ]:
# Loading Model: download if doesn't exist in current directory
model = keras.models.load_model('path/to/location')

In [38]:
file_testing = 'data/testing_data.txt'
file_prediction = 'prediction.csv'

In [45]:
with open(file_testing, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    text = []
    for line in lines[1:]:
        parsed = line.split(',', maxsplit=1)
        text.append(parsed[1].rstrip('\n'))
        
    data_testing = pd.DataFrame({'text': text})

In [ ]:
model_best.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model_best(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)

In [46]:
# Write prediction results into a csv file
with open("predict.csv", 'w') as f:
    f.write('id,label\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))

,text
0,"my dog ate our dinner . no , seriously ... he ..."
1,omg last day sooon n of primary noooooo x im g...
2,stupid boys .. they ' re so .. stupid !
3,hi ! do u know if the nurburgring is open for ...
4,"having lunch in the office , and thinking of h..."
5,shopping was fun
6,wondering where all the nice weather has gone .
7,morning ! yeeessssssss new mimi in aug
8,umm ... maybe that ' s how the british spell it ?
9,yes it ' s 3 : 50 am . yes i ' m still awake ....
